In [1]:
import Foundation
import TensorFlow

In [2]:
let trainCSV = try String(contentsOfFile:"../data/train.csv", encoding: String.Encoding.utf8)
let testCSV = try String(contentsOfFile:"../data/test.csv", encoding: String.Encoding.utf8)

let trainRecords: [[Float]] = trainCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }
let testRecords: [[Float]] = testCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }

let numTrainRecords = trainRecords.count
let numTrainColumns = trainRecords[0].count
let numTestRecords = testRecords.count
let numTestColumns = testRecords[0].count

print(numTrainRecords, numTrainColumns, numTestRecords, numTestColumns)

let xTrain = trainRecords.map{ Array($0[0..<numTrainColumns-1]) }
let yTrain = trainRecords.map{ [$0[numTrainColumns-1]] }
let xTest = testRecords.map{ Array($0[0..<numTestColumns-1]) }
let yTest = testRecords.map{ [$0[numTestColumns-1]] }

print(xTrain.count, xTrain[0].count, yTrain.count, yTrain[0].count,
      xTest.count, xTest[0].count, yTest.count, yTest[0].count)

let xAllTrain = Array(xTrain.joined())
let yAllTrain = Array(yTrain.joined())
let xAllTest = Array(xTest.joined())
let yAllTest = Array(yTest.joined())

print(xAllTrain.count, yAllTrain.count, xAllTest.count, yAllTest.count)

405 14 101 14
405 13 405 1 101 13 101 1
5265 405 1313 101


In [3]:
let XTrain = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numTrainColumns-1]))
let YTrain = Tensor<Float>(yAllTrain)
let XTest = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numTestColumns-1]))
let YTest = Tensor<Float>(yAllTest)

print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405] [101, 13] [101]


In [4]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

In [5]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [6]:
for _ in 0..<1000 {
    let 𝛁model = model.gradient { r -> Tensor<Float> in
        let ŷ = r(XTrain)
        let loss = meanSquaredError(predicted: ŷ, expected: YTrain)
        print("Loss: \(loss)")
        return loss
    }
    optimizer.update(&model, along: 𝛁model)
}

Loss: 604.4277
Loss: 593.51636
Loss: 585.7966
Loss: 579.3233
Loss: 573.486
Loss: 568.0008
Loss: 562.7851
Loss: 557.70764
Loss: 552.6743
Loss: 547.7055
Loss: 542.7961
Loss: 537.951
Loss: 533.11725
Loss: 528.3087
Loss: 523.5155
Loss: 518.7375
Loss: 513.91437
Loss: 509.06757
Loss: 504.22372
Loss: 499.40027
Loss: 494.59094
Loss: 489.77972
Loss: 484.9583
Loss: 480.1252
Loss: 475.28183
Loss: 470.42206
Loss: 465.5454
Loss: 460.6417
Loss: 455.70224
Loss: 450.72345
Loss: 445.71255
Loss: 440.6655
Loss: 435.57404
Loss: 430.4449
Loss: 425.2821
Loss: 420.08575
Loss: 414.85724
Loss: 409.60434
Loss: 404.329
Loss: 399.03064
Loss: 393.7015
Loss: 388.34515
Loss: 382.96805
Loss: 377.5761
Loss: 372.17487
Loss: 366.76825
Loss: 361.36542
Loss: 355.9687
Loss: 350.5769
Loss: 345.19376
Loss: 339.82382
Loss: 334.46783
Loss: 329.12973
Loss: 323.81317
Loss: 318.5189
Loss: 313.2489
Loss: 308.00827
Loss: 302.80176
Loss: 297.63004
Loss: 292.49615
Loss: 287.40277
Loss: 282.3514
Loss: 277.35004
Loss: 272.40283
Loss: 2

Loss: 80.04007
Loss: 80.03131
Loss: 80.02496
Loss: 80.02232
Loss: 80.02132
Loss: 80.02367
Loss: 80.02939
Loss: 80.04262
Loss: 80.06355
Loss: 80.08544
Loss: 80.07938
Loss: 80.05827
Loss: 80.03358
Loss: 80.019516
Loss: 80.01031
Loss: 80.005905
Loss: 80.00349
Loss: 80.003815
Loss: 80.00807
Loss: 80.02449
Loss: 80.06844
Loss: 80.11735
Loss: 80.087654
Loss: 80.04597
Loss: 80.014305
Loss: 80.0019
Loss: 79.9969
Loss: 79.99905
Loss: 80.00919
Loss: 80.02795
Loss: 80.045456
Loss: 80.04115
Loss: 80.02795
Loss: 80.012184
Loss: 80.00524
Loss: 80.004234
Loss: 80.01255
Loss: 80.02796
Loss: 80.04776
Loss: 80.04981
Loss: 80.03853
Loss: 80.016235
Loss: 80.00212
Loss: 79.99127
Loss: 79.98614
Loss: 79.98311
Loss: 79.98343
Loss: 79.98621
Loss: 79.99499
Loss: 80.011086
Loss: 80.03596
Loss: 80.04603
Loss: 80.03463
Loss: 80.00827
Loss: 79.99085
Loss: 79.978806
Loss: 79.97273
Loss: 79.96914
Loss: 79.96811
Loss: 79.96892
Loss: 79.97394
Loss: 79.98945
Loss: 80.03732
Loss: 80.095184
Loss: 80.0756
Loss: 80.010216
